In [ ]:
import re

# Define the file path
file_path = '/content/reddit_short_stories.txt'

# Read the file
with open(file_path, 'r') as file:
    content = file.read()

# Define pattern to remove unwanted data before <eos>
# Includes keywords like subscribe, PART, etc.
import re

# Define unwanted pattern
unwanted_pattern = r'<nl>.*?(\[.*?\]\(http[s]?://\S+\)|subscribe|/r/|PART \d+).*?<eos>'

# Apply the regex to remove unwanted data
cleaned_content = re.sub(unwanted_pattern, '<eos>', content, flags=re.IGNORECASE)


# Define the output file path
output_file_path = '/content/cleaned_reddit_short_stories.txt'

# Save the cleaned content back to a file
with open(output_file_path, 'w') as file:
    file.write(cleaned_content)

output_file_path


'/content/cleaned_reddit_short_stories.txt'

In [ ]:
!pip install transformers datasets torch accelerate


In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load your cleaned dataset
file_path = "/content/cleaned_reddit_short_stories.txt"
dataset = load_dataset("text", data_files={"train": file_path})

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS for compatibility


def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # Set labels as input_ids
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4308 [00:00<?, ? examples/s]

In [ ]:
sample_text = "This is a sample story. <eos>"
tokens = tokenizer.encode(sample_text, add_special_tokens=False)
decoded_text = tokenizer.decode(tokens)
print(f"Original: {sample_text}")
print(f"Decoded: {decoded_text}")

Original: This is a sample story. <eos>
Decoded: This is a sample story. <eos>


In [ ]:
# Split dataset into 90% training and 10% validation
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


In [ ]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust for tokenizer changes

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
import torch
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-reddit-stories",  # Output directory
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=2,
    fp16=True if torch.cuda.is_available() else False,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-11-dda2bb812d2b>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.292500,2.288989
2,2.260600,2.279329
3,2.383300,2.278890


TrainOutput(global_step=1455, training_loss=2.3395948836074254, metrics={'train_runtime': 27456.8186, 'train_samples_per_second': 0.424, 'train_steps_per_second': 0.053, 'total_flos': 3039087624192000.0, 'train_loss': 2.3395948836074254, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_save_path = "/content/drive/My Drive/gpt2-fine-tuned"

In [ ]:
# Save to Google Drive
!cp -r gpt2-fine-tuned /content/drive/MyDrive/

In [ ]:
model.save_pretrained("./gpt2-fine-tuned")
tokenizer.save_pretrained("./gpt2-fine-tuned")


('./gpt2-fine-tuned/tokenizer_config.json',
 './gpt2-fine-tuned/special_tokens_map.json',
 './gpt2-fine-tuned/vocab.json',
 './gpt2-fine-tuned/merges.txt',
 './gpt2-fine-tuned/added_tokens.json')

In [ ]:
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 2.2788896560668945, 'eval_runtime': 283.9405, 'eval_samples_per_second': 1.518, 'eval_steps_per_second': 0.19, 'epoch': 3.0}


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2-fine-tuned", tokenizer=tokenizer)
output = generator("Once upon a time,", max_length=100, num_return_sequences=1)
print(output[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Once upon a time, I've been making up my mind about "what's happening" in the universe. So, what do you expect? Are we even here, that we have "space"? The universe may be moving. This is why I have a great need for science. There's nothing you could do to alter it and even though it seems strange and frightening enough, that's what I have in common with those people who think in the same way this can happen to other people. The


Once upon a time, I've been making up my mind about "what's happening" in the universe. So, what do you expect? Are we even here, that we have "space"? The universe may be moving. This is why I have a great need for science. There's nothing you could do to alter it and even though it seems strange and frightening enough, that's what I have in common with those people who think in the same way this can happen to other people. The

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2-fine-tuned", tokenizer=tokenizer)
output = generator("All my life", max_length=100, num_return_sequences=1)
print(output[0]["generated_text"])

All my life, it just seemed like a bad idea before I saw the picture. A picture, to be exact? I had never seen one before as a child. And like an angry, confused child, I always wondered about it. I wanted to tell it to her. To be the first to have seen her, not her, but me too, who would say I deserved her? But no, I couldn't.

"This is...it really is," I said solemnly


All my life, it just seemed like a bad idea before I saw the picture. A picture, to be exact? I had never seen one before as a child. And like an angry, confused child, I always wondered about it. I wanted to tell it to her. To be the first to have seen her, not her, but me too, who would say I deserved her? But no, I couldn't.

"This is...it really is," I said solemnly